1 Create a Logging Table (Delta)

In [0]:
from pyspark.sql import Row
from pyspark.sql import functions as F
from datetime import datetime

log_path = "/Volumes/project4cat/project4db/p4gold/pipeline_logs"


In [0]:
spark.createDataFrame([], "stage STRING, status STRING, records_in LONG, records_out LONG, run_time STRING, timestamp TIMESTAMP") \
    .write.format("delta").mode("overwrite").save(log_path)


2 Create a Logging Function

In [0]:
# This function will log:
# Stage name
# Status (SUCCESS/FAILED)
# Input records
# Output records
# Run time
# Timestamp

def log_event(stage, status, records_in=None, records_out=None, run_time=None):
    log_df = spark.createDataFrame([
        Row(
            stage=stage,
            status=status,
            records_in=records_in,
            records_out=records_out,
            run_time=run_time,
            timestamp=datetime.now()
        )
    ])
    
    log_df.write.format("delta").mode("append").save(log_path)


3 Add Logging to Bronze Layer

In [0]:
start = datetime.now()

df_raw = spark.read.option("header","true").csv(raw_path)
records_in = df_raw.count()

df_bronze = df_raw.withColumn("ingestion_timestamp", F.current_timestamp())
records_out = df_bronze.count()

df_bronze.write.format("delta").mode("overwrite").save(bronze_path)

end = datetime.now()
run_time = str(end - start)

log_event("BRONZE_RETAIL", "SUCCESS", records_in, records_out, run_time)
